In [1]:
import pandas as pd
import json
import ast
import warnings
from ast import literal_eval as le
import plotly.express as px

warnings.filterwarnings("ignore")
import requests
import pandas as pd
import urlexpander


def unshorten(url):
    if "http" not in url:
        url = "http://" + url

    try:
        short = urlexpander.expand(url)
    except:
        short = url

    if "_ERROR__" in short:
        return url
    else:
        return short


def one_url_df(df, column="n_urls", urls="urls"):
    df = df[df[column] == 1]
    df[urls] = df[urls].apply(lambda x: x[0])
    return df


def more_url_df(df, interaction, n="n_urls", urls="urls"):
    df = df[df[n] > 1]
    temp = []
    for i in df.iterrows():
        row = i[1].to_dict()
        for url in row[urls]:
            temp.append((url, row[interaction], row[n]))
    return (
        pd.DataFrame(temp)
        .drop_duplicates()
        .rename(columns={0: urls, 1: interaction, 2: n})
    )

def plot(df, x):
    fig = px.histogram(df, x)
    fig.show();
    
def fix(df):
    df["urls"] = df["urls"].apply(lambda x: le(x))
    df["n_urls"] = df["urls"].apply(lambda x: len(x))

def un(df):
    df["urls"] = df["urls"].apply(lambda x: unshorten(x))
    
def sort(df, column):
    df.sort_values(column, inplace=True, ascending=False)

# Top Shared

## Top Interacted

In [5]:
top_4_months = pd.read_csv("../Top_Interacted/Data/reddits_4_months.csv")
top_4_months.rename(columns={"media_id": "id/url"}, inplace=True)
top_4_months.head(5)

,shared_url,id/url,text,score,total
0,https://www.fairfaxcounty.gov/circuit/sites/ci...,t1_gut3l5w,AH subpoenaed this footage from the LAPD herse...,1203,1203
1,https://newstaco.com/2011/08/30/how-raul-julia...,t3_kyj460,This man should be charged with paternity frau...,166,166
2,https://www.forbes.com/sites/keithflamer/2021/...,t3_lp629y,HOLD THE FUCKING PHONE:\n\nThis website is a j...,154,154
3,https://www.nytimes.com/2005/09/18/movies/six-...,t1_gjgw156,"upon researching, this article in the New York...",132,132
4,https://www.forbes.com/sites/keithflamer/2021/...,t3_lp629y,Forbes is already reporting this as genuine:\n...,129,129


In [6]:
top_dec = pd.read_csv("../Top_Interacted/Data/reddits_dec.csv")
top_dec.rename(columns={"media_id": "id/url"}, inplace=True)
top_dec.head(5)

,shared_url,id/url,text,score,total
0,https://youtu.be/uretrfMA-Io,t3_kiq88y,https://youtu.be/uretrfMA-Io\n\nEdit: 88k down...,618,618
1,https://twitter.com/AM59193334/status/13369634...,t1_gg0u0oa,"""Depp alleges that she was so angry she threw...",127,127
2,https://capcom.fandom.com/wiki/Zangief,t1_gg0fc4t,"158kg of pure beef, that's who. https://capcom...",126,126
3,https://m.huffpost.com/us/entry/us_5d8d58c5e4b...,t1_gfyhv2h,She is an attractive feminist woman. UN Women...,101,101
4,https://www.youtube.com/watch?v=t2HJB8M5lD4,t1_gg0j76i,lol there's literally pictures of Elon with br...,83,83


## Shared URLs

In [7]:
shared_4_months = pd.read_csv("../URLs_Data/df_shared_4_months.csv")
shared_4_months.fillna("{1}", inplace=True)
shared_4_months["n_urls"] = shared_4_months["n_urls"].apply(lambda x: le(x))
shared_4_months.rename(columns={"shared_link": "shared_url"}, inplace=True)
shared_4_months.head(5)

,shared_url,n_shared,from_n_users,users,n_urls,platform,from
0,https://www.change.org/p/walt-disney-back-john...,6,1,"(1181480925,)",{1},change,petition_comment
1,https://www.change.org/p/me-johnny-depp-as-jac...,3,1,"(1181480925,)",{1},change,petition_comment
2,https://www.change.org/p/%D0%BC%D0%B8%D0%B4-%D...,2,1,{1179297829},{4},change,petition_comment
3,https://www.change.org/p/loma-vista-recordings...,2,1,{1179297829},{4},change,petition_comment
4,https://www.change.org/p/warner-brothers-keep-...,2,1,{1198070350},{2},change,petition_comment


In [8]:
shared_dec = pd.read_csv("../URLs_Data/df_shared_dec.csv")
shared_dec.rename(columns={"shared_link": "shared_url"}, inplace=True)
shared_dec.head(5)

,shared_url,n_shared,from_n_users,users,n_urls,platform,from
0,https://www.documentcloud.org/documents/583586...,16,4,"{948054032, 1129808400, 1136503772, 1081693988}","{8, 10, 4}",change,petition_comment
1,https://www.latimes.com/entertainment/gossip/l...,16,4,"{948054032, 1129808400, 1136503772, 1081693988}","{8, 10, 4}",change,petition_comment
2,https://www.truehollywoodtalk.com/amber-heards...,10,2,"{1136503772, 1081693988}","{8, 10, 2}",change,petition_comment
3,https://www.youtube.com/channel/UCjKNc-woAlnxt...,8,2,"{1136503772, 1081693988}","{8, 10}",change,petition_comment
4,https://www.change.org/p/we-support-amber-hear...,6,1,{1081693988},{10},change,petition_comment


# 4

### Top shared reddit

In [9]:
shared_4_months_reddit = shared_4_months[shared_4_months.platform == "reddit"]
shared_4_months_reddit

,shared_url,n_shared,from_n_users,users,n_urls,platform,from
246,https://jerkofftocelebs.com/,139,1,{'AutoModerator'},"{4, 5, 6}",reddit,reddits
247,https://discord.gg/FMhrH2j,139,1,{'AutoModerator'},"{4, 5, 6}",reddit,reddits
248,https://jerkofftocelebs.com/pictures/,128,1,{'AutoModerator'},{6},reddit,reddits
249,https://jerkofftocelebs.com/actors/,128,1,{'AutoModerator'},{6},reddit,reddits
250,https://jerkofftocelebs.com/reddit-nsfw-list/,128,1,{'AutoModerator'},{6},reddit,reddits
...,...,...,...,...,...,...,...
908,http://www.reddit.com/message/compose?to=%2Fr%...,1,1,{'AutoModerator'},{2},reddit,reddits
909,https://www.reddit.com/r/FamosasGostosas,1,1,{'AutoModerator'},{4},reddit,reddits
910,https://www.reddit.com/r/Rebolando_,1,1,{'AutoModerator'},{4},reddit,reddits
911,https://www.reddit.com/r/FamosasGostosas/w/rules,1,1,{'AutoModerator'},{4},reddit,reddits


In [12]:
saving = "Data/"

In [13]:
top_shared_reddit_4_months = pd.merge(
    shared_4_months_reddit, top_4_months, how="inner"
)
top_shared_reddit_4_months = top_shared_reddit_4_months.sort_values(
    "total", ascending=False
).reset_index(drop=True)

In [14]:
top_shared_reddit_4_months.to_csv(saving + "top_shared_reddits_4_months.csv")

In [15]:
top_shared_reddit_4_months

,shared_url,n_shared,from_n_users,users,n_urls,platform,from,id/url,text,score,total
0,https://www.fairfaxcounty.gov/circuit/sites/ci...,2,1,"('possessedbyelf',)",{1},reddit,reddits,t1_gut3l5w,AH subpoenaed this footage from the LAPD herse...,1203,1203
1,https://newstaco.com/2011/08/30/how-raul-julia...,1,1,"('AltruisticVariation4',)",{1},reddit,reddits,t3_kyj460,This man should be charged with paternity frau...,166,166
2,https://www.forbes.com/sites/keithflamer/2021/...,1,1,"('jostler57',)",{1},reddit,reddits,t3_lp629y,HOLD THE FUCKING PHONE:\n\nThis website is a j...,154,154
3,https://www.nytimes.com/2005/09/18/movies/six-...,3,3,"('Spicy_Nana', 'read_listen_think', 'WordsinaW...",{1},reddit,reddits,t1_gjgw156,"upon researching, this article in the New York...",132,132
4,https://www.forbes.com/sites/keithflamer/2021/...,2,1,"('NotherThrowaway284',)",{1},reddit,reddits,t3_lp629y,Forbes is already reporting this as genuine:\n...,129,129
...,...,...,...,...,...,...,...,...,...,...,...
215,https://www.lamag.com/culturefiles/johnny-depp...,2,1,"('lilbobblehead',)",{1},reddit,reddits,t1_gut8lmf,https://www.lamag.com/culturefiles/johnny-depp...,3,3
216,https://youtu.be/1khEVKpz6Qw,2,2,"('driverishere', '_Pupa')",{1},reddit,reddits,t3_kqbbju,[Here](https://youtu.be/1khEVKpz6Qw),3,3
217,https://www.sausageroll.com.au/lifestyle/celeb...,2,1,{'dhalsim282'},{2},reddit,reddits,t3_kpm1c1,"So since this is just a meme, I searched to se...",3,3
218,https://www.sausageroll.com.au/lifestyle/celeb...,2,1,{'dhalsim282'},{2},reddit,reddits,t3_kpm1c1,"So since this is just a meme, I searched to se...",3,3


### Top shared reddit across all platforms

In [16]:
shared_4_months_reddit_all_platforms = pd.merge(
    shared_4_months, top_4_months, how="inner"
)
top_shared_reddit_4_months = top_shared_reddit_4_months.sort_values(
    "total", ascending=False
).reset_index(drop=True)

In [17]:
shared_4_months_reddit_all_platforms

,shared_url,n_shared,from_n_users,users,n_urls,platform,from,id/url,text,score,total
0,https://www.google.com/amp/s/www.dailymail.co....,2,1,"(7321139930.0,)",{1},instagram,comment,t1_gplgwll,https://www.google.com/amp/s/www.dailymail.co....,15,15
1,https://www.google.com/amp/s/www.dailymail.co....,2,1,"(7321139930.0,)",{1},instagram,comment,t1_gsk5k90,"Eye for an eye now? They're both not perfect, ...",5,5
2,https://www.google.com/amp/s/www.dailymail.co....,3,3,"('lesbanems', 'CrustyJohnson', 'XXAzeritsXx')",{1},reddit,reddits,t1_gplgwll,https://www.google.com/amp/s/www.dailymail.co....,15,15
3,https://www.google.com/amp/s/www.dailymail.co....,3,3,"('lesbanems', 'CrustyJohnson', 'XXAzeritsXx')",{1},reddit,reddits,t1_gsk5k90,"Eye for an eye now? They're both not perfect, ...",5,5
4,https://www.reddit.com/r/AmputatorBot/comments...,27,1,{'AmputatorBot'},"{8, 6}",reddit,reddits,t1_guuru7p,It looks like you shared some AMP links. These...,3,3
...,...,...,...,...,...,...,...,...,...,...,...
283,https://www.thegeekbuzz.com/the-basement/video...,2,2,"(9.1815297172838e+17, 80546748.0)",{1},twitter,tweet,t3_lx2s7w,"Do you ever feel like you have been lied to, b...",10,10
284,https://www.thegeekbuzz.com/the-basement/video...,1,1,{'UCEAomcLuClCPT-HRB-KBu_w'},{4},youtube,comment,t3_lx2s7w,"Do you ever feel like you have been lied to, b...",10,10
285,https://www.bbc.com/news/world-asia-india-3426...,1,1,{'edit_aword'},{2},reddit,reddits,t1_gutfh05,Gandhi was a reluctant apartheid supporter. H...,4,4
286,https://www.google.com/amp/s/www.independent.c...,1,1,{'edit_aword'},{2},reddit,reddits,t1_gutfh05,Gandhi was a reluctant apartheid supporter. H...,4,4


In [18]:
shared_4_months_reddit_all_platforms.to_csv(saving + "top_shared_4_months_reddits_all_platforms.csv")

# DEC

### Top shared instagram

In [19]:
shared_dec_reddit = shared_dec[shared_dec.platform == "reddit"]
shared_dec_reddit

,shared_url,n_shared,from_n_users,users,n_urls,platform,from
176,https://jerkofftocelebs.com/,36,1,{'AutoModerator'},"{5, 6}",reddit,reddits
177,https://discord.gg/FMhrH2j,36,1,{'AutoModerator'},"{5, 6}",reddit,reddits
178,https://jerkofftocelebs.com/actors/,35,1,{'AutoModerator'},"{5, 6}",reddit,reddits
179,https://jerkofftocelebs.com/pictures/,35,1,{'AutoModerator'},"{5, 6}",reddit,reddits
180,https://jerkofftocelebs.com/reddit-nsfw-list/,35,1,{'AutoModerator'},"{5, 6}",reddit,reddits
...,...,...,...,...,...,...,...
627,https://www.merriam-webster.com/dictionary/cri...,1,1,{'RetreadRoadRocket'},{2},reddit,reddits
628,https://www.merriam-webster.com/dictionary/con...,1,1,{'RetreadRoadRocket'},{2},reddit,reddits
629,https://meteamedia.org/17260/spotlight/toxic-m...,1,1,{'Danxe12'},{3},reddit,reddits
630,https://anzmh.asn.au/mental-health/mens-mental...,1,1,{'Danxe12'},{3},reddit,reddits


In [20]:
top_shared_reddit_dec = pd.merge(
    shared_dec_reddit, top_dec, how="inner"
)
top_shared_reddit_dec = top_shared_reddit_dec.sort_values(
    "total", ascending=False
).reset_index(drop=True)

In [21]:
top_shared_reddit_dec

,shared_url,n_shared,from_n_users,users,n_urls,platform,from,id/url,text,score,total
0,https://youtu.be/uretrfMA-Io,7,6,"('n4pth4', 'Snow_blind1211', 'mastercontrovers...",{1},reddit,reddits,t3_kiq88y,https://youtu.be/uretrfMA-Io\n\nEdit: 88k down...,618,618
1,https://youtu.be/uretrfMA-Io,5,3,"{'CaptainPrestedge', 'kflave249', '-banned-'}",{2},reddit,reddits,t3_kiq88y,https://youtu.be/uretrfMA-Io\n\nEdit: 88k down...,618,618
2,https://twitter.com/AM59193334/status/13369634...,2,1,"('zeixble',)",{1},reddit,reddits,t1_gg0u0oa,"""Depp alleges that she was so angry she threw...",127,127
3,https://capcom.fandom.com/wiki/Zangief,1,1,"('Aphaelo',)",{1},reddit,reddits,t1_gg0fc4t,"158kg of pure beef, that's who. https://capcom...",126,126
4,https://m.huffpost.com/us/entry/us_5d8d58c5e4b...,1,1,"('duhhhh',)",{1},reddit,reddits,t1_gfyhv2h,She is an attractive feminist woman. UN Women...,101,101
...,...,...,...,...,...,...,...,...,...,...,...
154,https://www.reddit.com/r/awfuleverything/comme...,1,1,"('MTG_Ginger',)",{1},reddit,reddits,t1_gg1oqze,"I don't mean to be rude Tweek, but your commen...",3,3
155,https://www.youtube.com/watch?v=Cg9SvQSMnoE&t=6s,1,1,{'jonnytechno'},{3},reddit,reddits,t1_ggswxx7,She punched him in the face a few times too .....,3,3
156,https://www.hollywoodreporter.com/features/hes...,2,1,{'HonorableJudgeIto'},{2},reddit,reddits,t1_gg0upye,"Yeah, but the other side of this story is that...",3,3
157,https://www.hollywoodreporter.com/features/hes...,2,1,{'HonorableJudgeIto'},{2},reddit,reddits,t1_gg0upye,"Yeah, but the other side of this story is that...",3,3


In [22]:
top_shared_reddit_dec.to_csv(saving + "top_shared_reddit_dec.csv")

### Top shared instagram across all platforms

In [23]:
shared_dec_reddit_all_platforms = pd.merge(
    shared_dec, top_dec, how="inner"
)
top_shared_reddit_dec = top_shared_reddit_dec.sort_values(
    "total", ascending=False
).reset_index(drop=True)

In [24]:
top_shared_reddit_dec

,shared_url,n_shared,from_n_users,users,n_urls,platform,from,id/url,text,score,total
0,https://youtu.be/uretrfMA-Io,7,6,"('n4pth4', 'Snow_blind1211', 'mastercontrovers...",{1},reddit,reddits,t3_kiq88y,https://youtu.be/uretrfMA-Io\n\nEdit: 88k down...,618,618
1,https://youtu.be/uretrfMA-Io,5,3,"{'CaptainPrestedge', 'kflave249', '-banned-'}",{2},reddit,reddits,t3_kiq88y,https://youtu.be/uretrfMA-Io\n\nEdit: 88k down...,618,618
2,https://twitter.com/AM59193334/status/13369634...,2,1,"('zeixble',)",{1},reddit,reddits,t1_gg0u0oa,"""Depp alleges that she was so angry she threw...",127,127
3,https://capcom.fandom.com/wiki/Zangief,1,1,"('Aphaelo',)",{1},reddit,reddits,t1_gg0fc4t,"158kg of pure beef, that's who. https://capcom...",126,126
4,https://m.huffpost.com/us/entry/us_5d8d58c5e4b...,1,1,"('duhhhh',)",{1},reddit,reddits,t1_gfyhv2h,She is an attractive feminist woman. UN Women...,101,101
...,...,...,...,...,...,...,...,...,...,...,...
154,https://www.hollywoodreporter.com/features/hes...,7,6,"('Chicken-n-Waffles', 'Baramos_', 'Poppin_Fres...",{1},reddit,reddits,t1_gg0upye,"Yeah, but the other side of this story is that...",3,3
155,https://change.org,1,1,{'TrifleAppropriate731'},{2},reddit,reddits,t3_kh64pm,I say its Danny Devito in a red wig. And I cre...,3,3
156,https://www.inquisitr.com/3164529/johnny-depp-...,1,1,{'LeanTangerine'},{2},reddit,reddits,t1_gf9laju,So I did more research and back in 2015-2016 t...,3,3
157,https://www.ibtimes.co.uk/amber-heard-charges-...,1,1,"('BunnyLovr',)",{1},reddit,reddits,t1_gg01dfg,https://www.ibtimes.co.uk/amber-heard-charges-...,3,3


In [25]:
shared_dec_reddit_all_platforms.to_csv(saving + "shared_dec_reddit_all_platforms.csv")

In [26]:
shared_dec_reddit_all_platforms

,shared_url,n_shared,from_n_users,users,n_urls,platform,from,id/url,text,score,total
0,https://www.change.org/p/dc-entertainment-remo...,1,1,"(948054032,)",{1},change,petition_comment,t1_gh94fnp,It fucking sucks that there is a need for this...,5,5
1,https://www.change.org/p/dc-entertainment-remo...,26,7,"(30074255940.0, 3703701611.0, 2200653740.0, 56...",{1},instagram,comment,t1_gh94fnp,It fucking sucks that there is a need for this...,5,5
2,https://www.change.org/p/dc-entertainment-remo...,2,2,"('SputnikSauce', 'Sociopathic-Panda')",{1},reddit,reddits,t1_gh94fnp,It fucking sucks that there is a need for this...,5,5
3,https://www.change.org/p/dc-entertainment-remo...,105,40,"(1.18243e+18, 8.95712e+17, 1.28422e+18, 1.2864...",{1},twitter,tweet,t1_gh94fnp,It fucking sucks that there is a need for this...,5,5
4,https://www.change.org/p/dc-entertainment-remo...,14,13,"{1.182426890956464e+18, 1.2850933833256673e+18...",{2},twitter,tweet,t1_gh94fnp,It fucking sucks that there is a need for this...,5,5
...,...,...,...,...,...,...,...,...,...,...,...
257,https://streamable.com/fy29r5,1,1,"('Dr-Satan-PhD',)",{1},reddit,reddits,t1_gg0di9k,[She's on tape admitting to abusing him](https...,8,8
258,https://en.wikipedia.org/wiki/Pirates\_,1,1,{'SgtFraggleRock'},{2},reddit,reddits,t3_k9pb24,Pretty sure that movie has already been made.\...,17,17
259,https://en.wikipedia.org/wiki/Pirates_,1,1,{'SgtFraggleRock'},{2},reddit,reddits,t3_k9pb24,Pretty sure that movie has already been made.\...,17,17
260,https://www.inquisitr.com/3164529/johnny-depp-...,1,1,{'LeanTangerine'},{2},reddit,reddits,t1_gf9laju,So I did more research and back in 2015-2016 t...,3,3


## Shared on Reddit

In [28]:
top_shared_reddit_4_months[["n_shared", "total"]].corr()

,n_shared,total
n_shared,1.000000,-0.034076
total,-0.034076,1.000000


In [29]:
top_shared_reddit_dec[["n_shared", "total"]].corr()

,n_shared,total
n_shared,1.00000,0.19396
total,0.19396,1.00000


## Shared across all platforms

In [30]:
shared_dec_reddit_all_platforms[["n_shared", "total"]].corr()

,n_shared,total
n_shared,1.000000,0.086171
total,0.086171,1.000000


In [31]:
shared_4_months_reddit_all_platforms[["n_shared", "total"]].corr()

,n_shared,total
n_shared,1.000000,-0.021894
total,-0.021894,1.000000
